In [2]:
import cv2
from ultralytics import YOLO
from collections import defaultdict
import numpy as np

In [4]:
# Load the YOLOv8 model
model = YOLO('model/best.pt')

# Open the video file
video_path = "multiple_rgby_01.mp4"
cap = cv2.VideoCapture(video_path)

# Store the track history
track_history = defaultdict(lambda: [])

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True)

        if results[0].boxes.id != None:
            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()

            # Plot the tracks
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 90 tracks for 90 frames
                    track.pop(0)

                # Draw the tracking lines
                points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
                cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=8)

            # Display the annotated frame
            cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 1 red_cube, 1 blue_cube, 1 green_cube, 1 yellow_cube, 178.3ms
Speed: 0.0ms preprocess, 178.3ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 red_cube, 1 blue_cube, 1 green_cube, 1 yellow_cube, 170.6ms
Speed: 15.6ms preprocess, 170.6ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 red_cube, 1 blue_cube, 1 green_cube, 1 yellow_cube, 171.0ms
Speed: 8.1ms preprocess, 171.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 red_cube, 1 blue_cube, 1 green_cube, 1 yellow_cube, 180.9ms
Speed: 0.0ms preprocess, 180.9ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 red_cube, 1 blue_cube, 1 green_cube, 1 yellow_cube, 180.8ms
Speed: 0.0ms preprocess, 180.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 red_cube, 1 blue_cube, 1 green_cube, 1 yellow_cube, 172.7ms
Speed: 8.1ms preprocess, 172.7ms inference, 8.0ms postprocess pe